In [ ]:
! pip3 install --user install fuzzywuzzy PyShp geopandas rtree matplotlib cartopy cython cartopy

In [10]:
import pandas as pd
import io
import requests
from fuzzywuzzy import process, fuzz
import shapefile
import zipfile
import json
import matplotlib.pyplot as plt
import geopandas as gpd
from cartopy import crs as ccrs

In [2]:
# read intermediate canonical facilities into dataframe
intermediate_url="https://raw.githubusercontent.com/PTDP/data/main/intermediate_data/intermediate_company_facilities.csv"
intermediate = pd.read_csv(intermediate_url)

hifld_url="https://opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
hifld=pd.read_csv(hifld_url)

In [3]:
# Normalize facility google name

# uppercase
# remove entities before dash IL DOC - BIG MUDDY RIVER CORRECTIONAL CENTER

intermediate['googlePlaceName'] = intermediate['googlePlaceName'].str.upper();

#print(intermediate)
#print(hifld)

In [4]:
#ratios = process.extract(intermediate['googlePlaceName'][0],hifld['NAME'])
# def fuzzy_matches(googlePlaceName):
    

# highest = process.extractOne(intermediate['googlePlaceName'][0],hifld['NAME'])
# print(highest)

# goal - 

# geosptial join

print(hifld['NAME'][1936])

CULLMAN COUNTY DETENTION CENTER


In [5]:
# Output GeoJSON Data
hifld_shp_url="https://opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0.zip?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
local_path = 'hifld.zip'

print('Downloading shapefile...')
r = requests.get(hifld_shp_url)

with open(local_path,'wb') as out: ## Open temporary file as bytes
    out.write(io.BytesIO(r.content).read())                ## Read bytes into file

# z = zipfile.ZipFile(io.BytesIO(r.content))
# z.write(local_path)
# z.close()
print("Done")
#z.extractall(path=local_path) # extract to folder
#filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
#print(filenames)

Done


In [43]:
#print(r.content)
# hifld_geojson = shapefile.Reader(local_path + 'Prison_Boundaries.dbf').__geo_interface__
gdf_intermediate = gpd.GeoDataFrame(intermediate, geometry=gpd.points_from_xy(intermediate.longitude, intermediate.latitude))

# remove null
gdf_intermediate = gdf_intermediate[gdf_intermediate.is_valid]
gdf_intermediate.crs = "EPSG:4326"
gdf_intermediate = gdf_intermediate.to_crs("EPSG:3857")

# buffer in miles around each geocoded point
buffer_length_in_meters = (5 * 1000) * 1.60934
gdf_intermediate['geometry'] = gdf_intermediate.geometry.buffer(buffer_length_in_meters)

# ax = gdf_intermediate.plot()

# distance + name similarity score

# fuzz.token_set_ratio(Str1,Str2)


gdf_hifld = gpd.read_file(local_path)

sjoined_facilities = gpd.sjoin(gdf_intermediate, gdf_hifld)
sjoined_facilities.head()
sjoined_facilities.count()


id                  10880
facilityInternal    10880
agencyInternal       6612
stateInternal       10880
company             10880
createdAt           10880
googlePlaceName     10880
address             10880
longitude           10880
latitude            10880
county              10790
googlePlaceId       10880
state               10880
geometry            10880
index_right         10880
FID                 10880
FACILITYID          10880
NAME                10880
ADDRESS             10880
CITY                10880
STATE               10880
ZIP                 10880
ZIP4                10880
TELEPHONE           10880
TYPE                10880
STATUS              10880
POPULATION          10880
COUNTY              10880
COUNTYFIPS          10880
COUNTRY             10880
NAICS_CODE          10880
NAICS_DESC          10880
SOURCE              10880
SOURCEDATE          10880
VAL_METHOD          10880
VAL_DATE            10880
WEBSITE             10880
SECURELVL           10880
CAPACITY    